## Top 10 Property Essentials Analysis

### Imports

In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
from nltk.corpus import stopwords
from nltk import ngrams
from dash import Dash, dcc, html, Input, Output
import plotly.express as px
import dash_bootstrap_components as dbc
from dash_bootstrap_templates import load_figure_template
from sklearn.cluster import KMeans
import os
os.environ['CURL_CA_BUNDLE'] = ''
from sentence_transformers import SentenceTransformer, util
import itertools
import warnings
import plotly.io as pio
warnings.filterwarnings("ignore")
#!pip install openpyxl -- uncomment if running for the first time
#also gotta install the other packages as well!

ValueError: 
Invalid named renderer(s) received: ['google-chrome']

### Converting xlsx file to csv's 

**If you have the CSV'a ignore the code and leave it commented out**  

**If csv's are outdated and you need to unpack an excel file, uncomment the block, replace the "excel_file" with the correct file path and run ONLY ONCE**

**once you have the csv's in the same directory, comment the block out and don't run again!**

In [7]:
# MAKE SURE THE "excel_file" IS IN THE SAME DIRECTORY!

# excel_file = 'importData/COMMENTARY.xlsx'
# all_sheets = pd.read_excel(excel_file, sheet_name=None)
# sheets = all_sheets.keys()

# for sheet_name in sheets:
#     sheet = pd.read_excel(excel_file, sheet_name=sheet_name)
#     sheet.to_csv("%s.csv" % sheet_name, index=False)

### Reading dataframes and Merging

In [2]:
attributes = pd.read_csv('importData/ATTRIBUTES.csv')
inspection = pd.read_csv('importData/INSPECTION.csv')
invoice = pd.read_csv('importData/INVOICE.csv')
question = pd.read_csv('importData/QUESTION.csv')
scores = pd.read_csv('importData/SCORES.csv')

### Code to extract month and year

In [3]:
def extract_month(date):
    date_list = date.split('-')
    month = int(date_list[1])
    return month

def extract_year(date):
    date_list = date.split('-')
    year = int(date_list[0])
    return year

In [4]:
inspection['MONTH'] = inspection['Created'].apply(extract_month)
inspection['YEAR'] = inspection['Created'].apply(extract_year)

### Creating bar chart of comments

In [5]:
def build_model(question, phrase_size, k):

    def create_dictionary(question, phrase_size):
        df = scores[scores['QUESTION'] == question]
        comments = df['COMMENT'].dropna()
        counts = {}
        for sentence in comments:
            lowercase = sentence.lower()
            cleaned = re.sub("[^\w\d'\s]+",'', lowercase).strip()
            if phrase_size == 1:
                words = cleaned.split()
                useless_words = stopwords.words("english")
                for word in words:
                    if word in counts.keys() and word not in useless_words:
                        counts[word] += 1
                    elif word not in counts.keys() and word not in useless_words:
                        counts[word] = 1
            else:
                pairs = list(ngrams(cleaned.split(), phrase_size))
                useless_words = set(stopwords.words('english'))
                for pair in pairs:
                    key = " ".join(pair)
                    if key in counts.keys():
                        counts[key] += 1
                    elif key not in counts.keys():
                        counts[key] = 1
        sorted_words = dict(sorted(counts.items(), key=lambda item: item[1], reverse=True))
        return sorted_words   

    def top_k_items(k, dictionary):
        top_items = dict(list(dictionary.items())[0:k])
        return top_items

    def top_k_dictionary(question, phrase_size, k):
        words = create_dictionary(question, phrase_size)
        sorted_dic = dict(sorted(words.items(), key=lambda item: item[1], reverse=True))
        top_items = top_k_items(2*k+5, sorted_dic)
        return top_items
    
    def create_embeddings(top_items):
        sentences = list(top_items.keys())
        counts = list(top_items.values())
        model = SentenceTransformer('distilbert-base-nli-mean-tokens')
        sentence_embeddings = model.encode(sentences)
        sentences_with_embeddings = list(zip(sentences, sentence_embeddings, counts))
        return sentences_with_embeddings
    
    def delete_similar_phrases(top_items, phrases_embeddings):
        sentences = list(top_items.keys())
        kept = dict.fromkeys(sentences, True)
        combos = list(itertools.combinations(phrases_embeddings, 2))
        for combo in combos:
            if kept[combo[0][0]] == False or kept[combo[1][0]] == False:
                continue
            similarity = util.pytorch_cos_sim(combo[0][1], combo[1][1]).item()
            if similarity >= 0.85:
                if combo[1][2] > combo[0][2]:
                    kept[combo[0][0]] = False
                else:
                    kept[combo[1][0]] = False
        return kept
    
    def reformat(reduced, original_dict):
        final_phrases = []
        for key, value in reduced.items():
            if value == True:
                final_phrases.append(key)
        filtered_dict = {}
        for phrase in final_phrases:
            filtered_dict[phrase] = original_dict[phrase]
        return filtered_dict
    
    top_items = top_k_dictionary(question, phrase_size, k)
    embeddings = create_embeddings(top_items)
    kept_dictionary = delete_similar_phrases(top_items, embeddings)
    reduced_dictionary = reformat(kept_dictionary, top_items)
    
    return reduced_dictionary

### Code for the App

In [18]:
app = Dash(__name__, external_stylesheets=[dbc.themes.DARKLY])
style = {'margin-left':'7px', 'margin-top':'7px'}
sidebar_style = {
    "position": "fixed",
    "top": 0,
    "left": 0,
    "bottom": 0,
    "width": "22rem",
    "padding": "1rem 1rem",
    "background-color": "black",
}

sidebar = html.Div([
        html.H2("Options"),
        html.Hr(),
        html.P(
            "Choose values for each option below", className="lead"
        ),
        dbc.Nav(
            [
                question_input := dcc.Dropdown(id='Select Question', options=scores['QUESTION'].unique(), value='Trees'),
                html.Br(),
                size_input := dcc.Input(type='number', value=3, min=1, max=10),
                html.Br(),
                k_input := dcc.Input(type='number', value=20, min=1, max=50)
            ],
            vertical=True,
            pills=True,
        ),
    ],
    style=sidebar_style,
)

app.layout = html.Div(children = [
                dbc.Row([
                    dbc.Col(),
                    dbc.Col(html.H1('Common Phrases per Reason'), width = 9, style = {'margin-left':'7px','margin-top':'7px'})
                    ]),
                dbc.Row(
                    [dbc.Col(sidebar),
                     dbc.Col(gr := dcc.Graph(id = 'graph2', figure = {}), width = 9, style = {'margin-left':'15px', 'margin-top':'7px', 'margin-right':'15px'})
                    ])
    ]
)

@app.callback(
    Output(gr, component_property='figure'),
    Input(question_input, 'value'),
    Input(size_input, 'value'),
    Input(k_input, 'value')
)
def update_graph(question_value, size_value, k):
    top_items = build_model(question_value, size_value, k)
    data = pd.DataFrame(list(top_items.items()), columns=['phrase', 'count'])
    fig = px.bar(data, x='phrase', y='count')
    #fig.show(renderer='browser')
    fig.update_layout(mapbox_style='carto-positron')
    return fig

load_figure_template('DARKLY')

if __name__ == '__main__':
    app.run_server(debug=False, jupyter_mode="external")

Dash app running on http://127.0.0.1:8050/


### Code that went into Power BI (it's a bit different in Power BI when accounting for the parameters)

In [48]:
#remember to add useless to the cell, it won't run in power bi!
def scratch(question, phrase_size, k):

    useless = set({'a','about','above','after','all',
        'an','and','any','are',"aren't",'as','at','be','because',
        'been','being','between','both','but','by','can',"couldn't",
        'did',"didn't",'do','does',"doesn't",'doing',"don't",'down',
        'during','each','few','for','from','had',"hadn't",'has',
        "hasn't",'have',"haven't",'having','he','her','here',
        'hers','herself','him','himself','his','how','i','if',
        'in','into','is',"isn't",'it',"it's",'its','just',
        'me',"mightn't",'more','my','myself',"needn't",
        'no','nor','not','now','of','off','on','once',
        'only','or','other','our','ours','ourselves','out',
        'over','own','same','she',"she's",'should',"should've",
        "shouldn't",'so','some','than','that','the','their',
        'theirs','them','then','there','these','they','this',
        'those','through','to','too','under','very','was',
        "wasn't",'we','were',"weren't",'what','when','where',
        'which','while','why','will','with',"won't","wouldn't"})

    def combinations(iterable, r):
        pool = tuple(iterable)
        n = len(pool)
        if r > n:
            return
        indices = list(range(r))
        yield tuple(pool[i] for i in indices)
        while True:
            for i in reversed(range(r)):
                if indices[i] != i + n - r:
                    break
            else:
                return
            indices[i] += 1
            for j in range(i+1, r):
                indices[j] = indices[j-1] + 1
            yield tuple(pool[i] for i in indices)

    def count_common(string1, string2):
        str1, str2 = string1.split(), string2.split()
        set1 = set(str1)
        common = len(set1.intersection(str2))
        # if ("nstb" in set1 or "ntb" in set1) and ("nstb" in str2 or "ntb" in str2):
        #     common += 1
        return common/len(str1)
    
    def clean(string):
        replacements = {
            'need to be': 'ntb',
            'needs to be': 'nstb'
        }
        for key, value in replacements.items():
            string = string.replace(key, value)
        return string

    def create_dictionary(question, phrase_size, useless):
        df = scores[scores['QUESTION'] == question]
        comments = df['COMMENT'].dropna()
        counts = {}
        for sentence in comments:
            lowercase = sentence.lower()
            cleaned = clean(re.sub("[^\w\d'\s]+",'', lowercase).strip())
            if phrase_size == 1:
                words = cleaned.split()
                for word in words:
                    if word in counts.keys() and word not in useless:
                        counts[word] += 1
                    elif word not in counts.keys() and word not in useless:
                        counts[word] = 1
            else:
                pairs = list(ngrams(cleaned.split(), phrase_size))
                for pair in pairs:
                    key = " ".join(pair)
                    if key in counts.keys():
                        counts[key] += 1
                    elif key not in counts.keys():
                        counts[key] = 1
        sorted_words = dict(sorted(counts.items(), key=lambda item: item[1], reverse=True))
        return sorted_words

    def top_k_items(k, dictionary):
        top_items = dict(list(dictionary.items())[0:k])
        return top_items

    def top_k_dictionary(question, phrase_size, k):
        words = create_dictionary(question, phrase_size, useless)
        sorted_dic = dict(sorted(words.items(), key=lambda item: item[1], reverse=True))
        top_items = top_k_items(2*k+5, sorted_dic)
        return top_items
    
    def delete_similar_phrases(top_items):
        tuples = list(top_items.items())
        phrases = list(map(lambda x: x[0], tuples))
        kept = dict.fromkeys(phrases, True)
        combos = list(combinations(tuples, 2))
        for combo in combos:
            if kept[combo[0][0]] == False or kept[combo[1][0]] == False:
                continue
            similarity = count_common(combo[0][0], combo[1][0])
            if similarity >= 0.66:
                if combo[1][1] > combo[0][1]:
                    kept[combo[0][0]] = False
                else:
                    kept[combo[1][0]] = False
        return kept
    
    def reformat(reduced, original_dict):
        final_phrases = []
        for key, value in reduced.items():
            if value == True:
                final_phrases.append(key)
        filtered_dict = {}
        for phrase in final_phrases:
            filtered_dict[phrase] = original_dict[phrase]
        return filtered_dict
    
    top_items = top_k_dictionary(question, phrase_size, k)
    kept_dictionary = delete_similar_phrases(top_items)
    reduced_dictionary = reformat(kept_dictionary, top_items)
    
    return reduced_dictionary